In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../..")

from heritageconnector import datastore
from heritageconnector.config import field_mapping
from heritageconnector.best_spacy_pipeline import load_model
from heritageconnector.datastore import es, index

import entity_linker

2021-02-15 10:13:08,436 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/


### NERLoader

In [19]:
source_description_field = target_description_field = "data.http://www.w3.org/2001/XMLSchema#description"
target_title_field = "graph.@rdfs:label.@value"
target_alias_field = "graph.@skos:altLabel.@value"
target_type_field = "graph.@skos:hasTopConcept.@value"

record_loader = datastore.RecordLoader("SMG", field_mapping)
ner_loader = datastore.NERLoader(
    record_loader,
    source_es_index = 'heritageconnector',
    target_es_index = 'heritageconnector',
    source_description_field = source_description_field, 
    target_title_field = target_title_field, 
    target_description_field = target_description_field, 
    target_type_field = target_type_field,
    target_alias_field = target_alias_field,
    entity_types_to_link={"PERSON", "OBJECT", "ORG"}
)

2021-02-15 10:39:32,094 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-02-15 10:39:32,094 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/
2021-02-15 10:39:32,094 - heritageconnector.datastore - DEBUG - Connected to Elasticsearch cluster at https://c4241473f2f84e2ab15c5b7c81eb34a4.eu-west-1.aws.found.io:9243/


In [27]:
entity_list = ner_loader.get_list_of_entities_from_es("en_core_web_trf", 500)
len(entity_list)

2021-02-15 10:48:12,591 - heritageconnector.datastore - INFO - Fetching docs and running NER.
2021-02-15 10:48:12,591 - heritageconnector.datastore - INFO - Fetching docs and running NER.
2021-02-15 10:48:12,591 - heritageconnector.datastore - INFO - Fetching docs and running NER.
2021-02-15 10:48:12,591 - heritageconnector.datastore - INFO - Fetching docs and running NER.
spacy tried to use GPU but failed
2021-02-15 10:48:16,879 - hc_nlp.pipeline - INFO - Loading thesaurus from ../../../heritageconnector/../GITIGNORE_DATA/labels_all_unambiguous_types_people_orgs.jsonl
2021-02-15 10:48:19,124 - hc_nlp.pipeline - INFO - 17016 term thesaurus imported in 2s
2021-02-15 10:48:19,127 - hc_nlp.pipeline - WARNING - Custom span attributes already added.


0it [00:00, ?it/s]

1406

In [28]:
entity_list_with_links = ner_loader.get_link_candidates(candidates_per_entity_mention=5)

2021-02-15 10:49:09,975 - heritageconnector.datastore - INFO - Getting link candidates for each of 1406 entities
2021-02-15 10:49:09,975 - heritageconnector.datastore - INFO - Getting link candidates for each of 1406 entities
2021-02-15 10:49:09,975 - heritageconnector.datastore - INFO - Getting link candidates for each of 1406 entities
2021-02-15 10:49:09,975 - heritageconnector.datastore - INFO - Getting link candidates for each of 1406 entities


  0%|          | 0/1406 [00:00<?, ?it/s]

In [29]:
review_df = ner_loader.get_links_data_for_review()
review_df
review_df.to_csv("./review_data.csv")

In [30]:
from spacy import displacy

In [33]:
displacy.render(ner_loader.nlp("Library of Congress Authorities:  n 93079091; http://encyclopedia.jrank.org/CAL_CAR/CANNIZZARO_STANISLAO_1826_1910_.html; JSTOR - Stanislao Cannizzaro, F.R.S. (1826-1910) and the First International Chemical Conference at Karlsruhe in 1860, by Hartley, Harold, Notes and Records of the Royal Society of London, Vol 21, No 1, Jun., 1966, pp. 56-63, (http://links.jstor.org/sici?sici=0035-9149%28196606%2921%3A1%3C56%3ASCF%28AT%3E2.0.CO%3B2-M) IN : 00359149; http://en.wikipedia.org/wiki/Stanislao_Cannizzaro "), style='ent')
